# On `*args, **kwargs`

Presentation to San Diego Python User Group on 23 January 2020

## Positional arguments vs keyword arguments

In [1]:
print("Hello", "world")

Hello world


In [2]:
print("Hello", "world", sep=", ")

Hello, world


### Rules

1.Positional arguments vs keyword arguments relates to how a function is *called*, not on how it is defined.

In [3]:
def product(x, y, z):
    return x * y * z

In [4]:
product(3, 5, 7) # using positional arguments

105

In [5]:
product(x=3, y=5, z=7) # using keyword arguments

105

2. Positional arguments must precede keyword arguments and 

In [6]:
product(x=3, 5, 7) # Doesn't work!

SyntaxError: positional argument follows keyword argument (<ipython-input-6-f65bc885876e>, line 1)

3. Positional arguments must appear in the order they appear in the function definition. 

In [7]:
product(3, 7, y=5) # Doesn't work!

TypeError: product() got multiple values for argument 'y'

4. Keyword arguments may appear in any order after the positional arguments.

In [8]:
product(3, z=7, y=5) # Does work!

105

5. In some cases, you don't have a choice. 

In [9]:
print('Hello', 'world', sep=', ') # sep must be a keyword argument

Hello, world


In [10]:
print('Hello', 'world', ', ')

Hello world , 


6. In the definition of a function, you can force an argument to always be a keyword argument. Such arguments are referred to as *keyword only arguments*.

In [11]:
def product(x, y, *, z):
    return x * y * z

In [12]:
product(4, 6, 8) # last argument cannot be positional

TypeError: product() takes 2 positional arguments but 3 were given

7. In some cases, mostly with builtin functions, an argument *must* be positional. In [Python 3.8](https://docs.python.org/3/whatsnew/3.8.html), it's possible to force an argument to be positional (so called *positional only argument*).

```Python
def product(x, /, y, z):
    return x * y * z
```

In [13]:
help(len)

Help on built-in function len in module builtins:

len(obj, /)
    Return the number of items in a container.



In [14]:
len(obj='abc')

TypeError: len() takes no keyword arguments

__Discussion:__ Why would one _force_ some arguments to be keyword or positional arguments?

### The unpacking operator `*`

In [15]:
def product(nums, start=1):
    for n in nums:
        start *= n
    return start

In [16]:
product([2, 3, 4])

24

Unlike `print`, which can take an arbitrary number of positional argument, the numbers to be multiplied by `product` have to be passed as one argument (an iterable). 

In [17]:
product(2, 3, 4) # Doesn't work

TypeError: product() takes from 1 to 2 positional arguments but 3 were given

To make the above work, we can use the `*` operator:

In [18]:
def product(*nums, start=1):
    # positional args are packed into a tuple nums
    for n in nums:
        start *= n
    return start

Think of `num` as the tuple containing the positional arguments that the function is called with, and `*nums` as the tuple without the enclosing parentheses. 

Note that, in a function *definition*, the `*` operator can only be applied to one argument, and that all following arguments must be keyword arguments.

In [19]:
product(2, 3, 4)

24

In [20]:
product(5, start=24)

120

In [21]:
product()

1

The \* operator can also be used to unpack a list (or other iterable) when *calling* a function:

In [22]:
num_list = [2, 3, 4]
product(*num_list)

24

In a function call, the `*` operator may be applied multiple times.

In [23]:
product(*num_list, *(5, 6))

720

**Exercise**

Re-write the following without using `join`

In [24]:
print(' + '.join(str(i) for i in range(1, 11))) 

1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10


In [26]:
# Solution
print(*range(1, 11), sep=' + ')

1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10


The unpacking operator can only be applied in a context where an arbitrary number of items is expected. By itself, it cannot be used.

In [27]:
*range(1, 11)

SyntaxError: can't use starred expression here (<ipython-input-27-fd83ffa90988>, line 4)

**Exercise**

Use the `*` operator to create the tuple of integers from 1 to 10 inclusive.

In [ ]:
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [29]:
# Solution
(*range(1, 11),)

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

**Exercise**
Use the `*` operator and `zip` to transpose the matrix `m`.

In [30]:
m = (*((*range(i+1, i+7),) for i in range(10, 40, 10)),)
print(m)

((11, 12, 13, 14, 15, 16), (21, 22, 23, 24, 25, 26), (31, 32, 33, 34, 35, 36))


In [32]:
# Solution
m_t = (*zip(*m),)
print(m_t)

((11, 21, 31), (12, 22, 32), (13, 23, 33), (14, 24, 34), (15, 25, 35), (16, 26, 36))


This goes to show that `zip` is essentially a transposition function.

### The dictionary unpacking operator `**`

In [34]:
d1 = {1:'a', 2:'b', 3:'c'}
d2 = {4:'d', 5:'e', 6:'f'}

In [35]:
{**d1} # copy a dictionary

{1: 'a', 2: 'b', 3: 'c'}

In [36]:
_ is d1

False

In [37]:
{**d1, 24:'x', 25:'y', 26:'z'} # copy and extend

{1: 'a', 2: 'b', 3: 'c', 24: 'x', 25: 'y', 26: 'z'}

In [38]:
{**d1, **d2} # combine two dicts into a third

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f'}

In [39]:
d1, d2

({1: 'a', 2: 'b', 3: 'c'}, {4: 'd', 5: 'e', 6: 'f'})

#### Using `**` when calling a function

In [40]:
def product(x, y, z):
    return x * y * z

In [41]:
nums = {'x':3, 'y': 4, 'z': 5}
product(**nums) # same as product(x=3, y=4, z=5)

60

This can be useful where the arguments are built one at the time before being passed to a function. Assume `setup_environment(home, docs, course)` is a function with the shown arguments. Then, one could do:

```Python
env = {}
env['home'] = '/Users/erikc'
env['docs'] = env['home'] + '/Documents'
env['course'] = env['docs'] + '/Learning/python_for_beginners'
...
# setup_environment(home=env['home'], docs=env['docs'], course=env['course'])
setup_environment(**env)
```

#### Using `**` when defining a function

Dict unpacking can be used to define functions that take an arbitrary list of keyword arguments. For example:

In [42]:
def print_scores(**scores):
    for name, score in scores.items():
        print(f"{name.replace('_', ' '):18}: {score}")            

In [43]:
print_scores(Alice_Appleseed=5.7, 
             Chris_Christensen=5.2, 
             Bob_Burlington=5.0, 
             Dan_Draper=4.9)            

Alice Appleseed   : 5.7
Chris Christensen : 5.2
Bob Burlington    : 5.0
Dan Draper        : 4.9


Notice that the arguments are processed in the order they passed in, which is a requirement on Python since Python 3.6. (Which is not obvious, since `scores` is a dict, and there is no inherent ordering of dict elements.)

## And finally, `*args, **kwargs`

In short, `*args` is simply a generic name for all the positional arguments that are passed in a function call, and `**kwargs` is a generic name for all the keyword arguments that are passed in a function call. (The naming is a convention; any names could be used, e.g., `*positional_arguments, **keyword_arguments`.) Any function can be called with at most two argument expressions; the first is a tuple (or any iterable) of all the positional arguments preceded by `*`, and the second is a dict of all the keyword arguments preceded by `**`. For example,

In [44]:
print('Hello', 'world', sep=', ', end='\n')
# Let's manually pattern match positional and keyword arguments
args = ('Hello', 'world')
kwargs = {'sep': ', ', 'end': '\n'}
print(*args, **kwargs) # same as print('Hello', 'world', sep=', ', end='\n')

Hello, world
Hello, world


Sometimes, e.g., when overriding a method or working with higher order functions, one needs to express _"whatever positional and keyword arguments are passed to `<some function>`"_. This often translates to code as `*args, **kwargs`.

The following example illustrates the use of `*args, **kwargs`. The function `log` takes a function `f` as argument and returns a new function, which, when called, prints the name of `f` and the positional and keyword arguments that it was called with, and then calls `f` with those same arguments. The function `log` provides a non-invasive (i.e., it doesn't require any modification of the function being logged) and reusable way of logging function calls. "

In [1]:
from time import ctime

def log(f):
    
    def h(*args, **kwargs):
        all_args = [repr(a) for a in args]
        all_args += [f"{k}={repr(v)}" for k, v in kwargs.items()]
        print(f"[Erik] {ctime()}: {f.__name__}({', '.join(all_args)})")
        return f(*args, **kwargs)

    return h

In [46]:
log(product)(1 + 2, z=17 - 15, y=2 + 3)

[Erik] Thu Jan 23 19:48:54 2020: product(3, z=2, y=5)


30

In [47]:
log(print)(*'abc', 'd', sep='_')

[Erik] Thu Jan 23 19:49:40 2020: print('a', 'b', 'c', 'd', sep='_')
a_b_c_d


#### Quick peek into decorators

We can think of the function `log` as something that modifies function *calls*. But we can also use it to modify function *definitions*.  When a higher order function is used to modify a function definition, it's called a *decorator*. Consider:

In [4]:
@log
def fib(n):
    if n < 2: return n
    return fib(n - 2) + fib(n - 1)

In [5]:
fib(5)

[Erik] Tue Feb  4 18:42:44 2020: fib(5)
[Erik] Tue Feb  4 18:42:44 2020: fib(3)
[Erik] Tue Feb  4 18:42:44 2020: fib(1)
[Erik] Tue Feb  4 18:42:44 2020: fib(2)
[Erik] Tue Feb  4 18:42:44 2020: fib(0)
[Erik] Tue Feb  4 18:42:44 2020: fib(1)
[Erik] Tue Feb  4 18:42:44 2020: fib(4)
[Erik] Tue Feb  4 18:42:44 2020: fib(2)
[Erik] Tue Feb  4 18:42:44 2020: fib(0)
[Erik] Tue Feb  4 18:42:44 2020: fib(1)
[Erik] Tue Feb  4 18:42:44 2020: fib(3)
[Erik] Tue Feb  4 18:42:44 2020: fib(1)
[Erik] Tue Feb  4 18:42:44 2020: fib(2)
[Erik] Tue Feb  4 18:42:44 2020: fib(0)
[Erik] Tue Feb  4 18:42:44 2020: fib(1)


5

## Summary

Three steps to understanding `*args, **kwargs`:

1. Understand the difference between positional and keyword arguments, and their usage
2. Understand how the unpacking operators `*` and `**` work
3. Realize that every function call _is_ on the form `f(*args, **kwargs)`